## Юнит 7. Основные алгоритмы машинного обучения. Часть II 
### Skillfactory: DSPR-19

### ML-5. Решающие деревья и случайный лес 

### 5.1. Введение
### 5.2. Решающие деревья

Формально дерево решений — это связный ациклический граф. В нём можно выделить три типа вершин:

- Корневая вершина (root node) — откуда всё начинается.
- Внутренние вершины (intermediate nodes).
- Листья (leafs) — самые глубокие вершины дерева, в которых содержится «ответ».

Во внутренней или корневой вершине признак проверяется на некий логический критерий, по результатам которого мы движемся всё глубже по дереву.

Определенный бонус решающего дерева в том, что его легко интерпретировать, а также легко визуализировать ход принятия решения. Однако есть и ряд недостатков. В целом достоинства и недостатки данного метода следующие:

**ДОСТОИНСТВА**

- Деревья требуют небольших усилий при подготовке данных (не требуется специфической предобработки).
- Дерево решений не требует нормализации/масштабирования данных.
- Наличие пропусков не оказывает существенного влияния на построение дерева.
- Модель деревьев решений достаточно интуитивно понятна и легко объяснима даже людям, не разбирающимся в методе.

**НЕДОСТАТКИ**

- Требует затрат больших вычислительных мощностей (по сравнению с другими алгоритмами).
- В силу дискретной структуры дерево не дифференцируется по параметрам — нельзя найти хотя бы локальный экстремум.
- Небольшое изменение в данных может заметно повлиять на структуру дерева.
- При работе с непрерывными числовыми признаками дерево делит их на категории, и теряет информацию.
- Алгоритм дает плохие результаты, если дерево используется для решения задачи классификации при несбалансированных классах.

**ИРИСЫ ФИШЕРА**